In [13]:
import pandas as pd

from math import cos, asin, sqrt, floor

# In km
def calcDistance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return round(12742 * asin(sqrt(a)), 2) #2*R*asin...

In [14]:
locations_df = pd.read_csv('../location/ip2location.csv').set_index('domain')

In [18]:
awsnv_lat = 38.747650
awsnv_lon = -77.532448

speedup = 3
adv_speed = (2.0/3)*299.792458
t_start = 5
t_tls_proc = 6.5
t_rt_proc = 2
t_client_proc = 1.5
t_porcom = 2
l = []

# print("domain | rtt_adv | rtt_honest | Delta | amplification factor | error (km)")
for domain in ["www.american.edu", "www.sunysuffolk.edu", "roughtime.chainpoint.org", "roughtime.sandbox.google.com", "roughtime.int08h.com"]:
    d = calcDistance(awsnv_lat, awsnv_lon, locations_df.at[domain, 'lat'], locations_df.at[domain, 'lon'])
    rtt_upper_bound = (2*d) / adv_speed
    rtt_lower_bound = rtt_upper_bound * speedup + t_start
    Delta = rtt_lower_bound + t_client_proc + t_porcom
    amp = 1
    extra = 0
    if (domain.startswith('roughtime')):
        Delta += t_rt_proc
    else:
        Delta += t_tls_proc
        amp = floor(1000 / Delta) - 1
        extra = (1000 - amp * Delta) / amp
        print(domain, extra)
    error = (Delta + extra) * adv_speed / 2
    l.append([domain, d, rtt_upper_bound, rtt_lower_bound, Delta, amp, error])

df = pd.DataFrame(l, columns=['domain', 'dist', 'rtt_adv', 'rtt_honest', 'Delta', 'a', 'radius (km)']).set_index('domain')
df.sort_values('Delta')

www.american.edu 0.34605305736766545
www.sunysuffolk.edu 0.8937128274725242


,dist,rtt_adv,rtt_honest,Delta,a,radius (km)
domain,,,,,,
roughtime.chainpoint.org,46.00,0.460318,6.380955,11.880955,1,1187.273603
roughtime.sandbox.google.com,115.33,1.154098,8.462295,13.962295,1,1395.263603
www.american.edu,43.99,0.440205,6.320614,16.320614,60,1665.513656
www.sunysuffolk.edu,450.29,4.506017,18.518052,28.518052,34,2939.141745
roughtime.int08h.com,1582.83,15.839258,52.517773,58.017773,1,5797.763603


In [19]:
# UK

uk_anchors = list(pd.read_csv('../uk-final.csv')['name'])

awslon_lat = 51.5085
awslon_lon = -0.1257

l = []
for domain in uk_anchors:
    d = calcDistance(awslon_lat, awslon_lon, locations_df.at[domain, 'lat'], locations_df.at[domain, 'lon'])
    rtt_upper_bound = (2*d) / adv_speed
    rtt_lower_bound = rtt_upper_bound * speedup + t_start
    Delta = rtt_lower_bound + t_client_proc + t_porcom
    amp = 1
    extra = 0
    if (domain.startswith('roughtime')):
        Delta += t_rt_proc
    else:
        Delta += t_tls_proc
        amp = floor(1000 / Delta) - 1
        extra = (1000 - amp * Delta) / amp
    error = (Delta + extra) * adv_speed / 2
    l.append([domain, d, rtt_upper_bound, rtt_lower_bound, Delta, amp, error])

df_uk = pd.DataFrame(l, columns=['domain', 'dist', 'rtt_adv', 'rtt_honest', 'Delta', 'a', 'radius (km)']).set_index('domain')
df_uk.sort_values('rtt_adv')

,dist,rtt_adv,rtt_honest,Delta,a,radius (km)
domain,,,,,,
holycross.ac.uk,35.26,0.352844,6.058532,16.058532,61,1638.210153
leeds.ac.uk,58.58,0.586206,6.758617,16.758617,58,1722.945161
sruc.ac.uk,58.83,0.588707,6.766122,16.766122,58,1722.945161
www.open.ac.uk,73.45,0.735008,7.205025,17.205025,57,1753.172269
gold.ac.uk,87.45,0.875105,7.625316,17.625316,55,1816.923988
enterpriseresearch.ac.uk,133.45,1.335424,9.006272,19.006272,51,1959.427830
le.ac.uk,143.24,1.433392,9.300175,19.300175,50,1998.616387
lboro.ac.uk,157.95,1.580593,9.741780,19.741780,49,2039.404476
mathcentre.ac.uk,157.95,1.580593,9.741780,19.741780,49,2039.404476


In [20]:
selected = ["holycross.ac.uk", "sruc.ac.uk", "gold.ac.uk", "nott.ac.uk", "www.ed.ac.uk"]
df_uk.loc[df_uk.index.isin(selected)].sort_values('Delta')

,dist,rtt_adv,rtt_honest,Delta,a,radius (km)
domain,,,,,,
holycross.ac.uk,35.26,0.352844,6.058532,16.058532,61,1638.210153
sruc.ac.uk,58.83,0.588707,6.766122,16.766122,58,1722.945161
gold.ac.uk,87.45,0.875105,7.625316,17.625316,55,1816.923988
nott.ac.uk,175.19,1.753113,10.259338,20.259338,48,2081.892069
www.ed.ac.uk,533.67,5.340395,21.021184,31.021184,31,3223.574817
